# Predictive Modeling of County-Level Acute Food Insecurity in Kenya

## Business Understanding

### Context and motivation

Acute food insecurity in Kenya fluctuates over time, particularly in arid and semi-arid areas (ASALs), creating a strong need for **early warning** rather than purely retrospective reporting to support timely actions such as cash transfers, market support, and preparedness logistics. Periodic assessments from the [IPC Kenya Country Analysis](https://www.ipcinfo.org/ipc-country-analysis/en/?country=KEN) and the [IPC Kenya Acute Food Insecurity Report (Jul 2025–Jan 2026)](https://www.ipcinfo.org/fileadmin/user_upload/ipcinfo/docs/IPC_Kenya_Acute_Food_Insecurity_Acute_Malnutrition_Jul2025_Jan2026_Report.pdf) consistently show that **Crisis or worse (IPC Phase 3+)** outcomes persist and expand during rainfall deficits and market stress. Forward-looking analyses from the [FEWS NET Kenya Food Security Outlook (Oct 2025)](https://fews.net/east-africa/kenya/food-security-outlook/october-2025) further demonstrate how rainfall performance and market conditions drive projected food security outcomes, while drought impact quantification from the [NDMA Long Rains Assessment Report 2025](https://knowledgeweb.ndma.go.ke/Content/LibraryDocuments/National_Long_Rains_Assessment_Report_202520250819170940.pdf) highlights deterioration patterns and populations at risk. At the global level, the [Global Report on Food Crises 2024](https://www.fightfoodcrises.net/report/global-report-food-crises-2024/) synthesizes evidence showing that conflict, weather extremes, and economic shocks remain the dominant drivers of acute food insecurity. Together, these sources reinforce the need for predictive, data-driven early warning systems that anticipate IPC Phase 3+ risk before households cross critical thresholds.

---

### Project goal
Build a **replicable county-level early-warning model** that predicts whether a county will be in **IPC Phase 3+ (Crisis or worse)** in the near future (e.g., 1–3 months ahead), using a small set of drivers that are available regularly and can be updated over time.


### Project objectives
1. Build a county-level model to predict IPC Phase 3+ food insecurity risk 1–3 months ahead.
2. Identify and interpret the key climate, market, conflict, and poverty drivers influencing acute food insecurity across Kenya.
3. Deliver a practical dashboard that presents forecasted risk and drivers to support timely humanitarian and policy decisions.

---

## Data Understanding

We use four data sources representing the target outcome and predictive signals:

### 1) IPC Kenya classifications (Target)
Historical IPC classifications by county and analysis period. We will derive a **county-month target** for **Phase 3+**.  

### 2) Subnational rainfall indicators (Predictors)
Monthly county rainfall totals, long-term averages, and anomalies. These capture **climate stress** (e.g., drought) that affects crops and livestock.

### 3) WFP food prices (Predictors)
Market-level staple food prices over time. These reflect **food access and market stress**.   

### 4) Multidimensional Poverty Index (MPI) (Predictors)
County-level socioeconomic vulnerability indicators. MPI is typically **static** (a snapshot), so we treat it as baseline features replicated across months for each county.  

---

### 1. Notebook Setup & Environment

In [1]:
# 1. Notebook Setup & Environment

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import warnings

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.4f}".format)

warnings.filterwarnings("ignore", category=FutureWarning)


### 2. Data Loading & Inventory

We load **four** datasets:

- **IPC** (target / labels)
- **Rainfall** (county-month climate features)
- **WFP food prices** (market-level prices; requires market → county mapping before county-month aggregation)
- **MPI** (county-level static vulnerability features)

We explicitly exclude the current conflict spreadsheet because it is **national-month only** (no county detail).


In [2]:
# 2. Data Loading & Inventory

@dataclass(frozen=True)
class DataPaths:
    ipc: Path
    rainfall: Path
    prices: Path
    mpi: Path

def resolve_data_dir() -> Path:
    """Resolve a data directory in a portable way.

    Priority:
    1) ./data (typical repo layout)
    2) /mnt/data (this environment / many notebooks)
    """
    cwd = Path.cwd()
    repo_data = (cwd.parent if cwd.name.lower() == "notebooks" else cwd) / "data"
    if repo_data.exists():
        return repo_data
    mnt = Path("/mnt/data")
    if mnt.exists():
        return mnt
    return repo_data  # fall back; existence checked below

DATA_DIR = resolve_data_dir()

paths = DataPaths(
    ipc=DATA_DIR / "ipc_ken_area_long.csv",
    rainfall=DATA_DIR / "kenya_county_rainfall_2018_2026.csv",
    prices=DATA_DIR / "wfp_food_prices_ken.csv",
    mpi=DATA_DIR / "ken_mpi.csv",
)

def read_csv_safely(path: Path) -> pd.DataFrame:
    """Read CSV with a safe fallback for encoding."""
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin-1")

missing = [p for p in [paths.ipc, paths.rainfall, paths.prices, paths.mpi] if not p.exists()]
if missing:
    print("Resolved DATA_DIR:", DATA_DIR)
    for p in [paths.ipc, paths.rainfall, paths.prices, paths.mpi]:
        print(f"- {p.name}: exists={p.exists()} -> {p}")
    raise FileNotFoundError(f"Missing required file(s): {[p.name for p in missing]}")

ipc_df = read_csv_safely(paths.ipc)
rain_df = read_csv_safely(paths.rainfall)
prices_df = read_csv_safely(paths.prices)
mpi_df = read_csv_safely(paths.mpi)

print("IPC:", ipc_df.shape)
print("Rainfall:", rain_df.shape)
print("Prices:", prices_df.shape)
print("MPI:", mpi_df.shape)


IPC: (4522, 11)
Rainfall: (4606, 10)
Prices: (17365, 16)
MPI: (49, 11)


### 3. Key Standardization & Alignment

In [6]:
# ============================================================
# SECTION 3A — IPC: Collapse "Area" into an official county field
# Goal: IPC "Area" mixes counties + sub-areas + settlements.
# Output: ipc_df with a clean `county` column (official 47 list).
# ============================================================

import re
import numpy as np
import pandas as pd

# --- Helpers ---
def clean_text(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
         .str.title()
    )

def to_month_start(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce")
    return dt.dt.to_period("M").dt.to_timestamp()

# --- Official Kenya counties (47) ---
KENYA_47 = [
    "Baringo","Bomet","Bungoma","Busia","Elgeyo-Marakwet","Embu","Garissa","Homa Bay","Isiolo",
    "Kajiado","Kakamega","Kericho","Kiambu","Kilifi","Kirinyaga","Kisii","Kisumu","Kitui",
    "Kwale","Laikipia","Lamu","Machakos","Makueni","Mandera","Marsabit","Meru","Migori",
    "Mombasa","Murang'A","Nairobi","Nakuru","Nandi","Narok","Nyamira","Nyandarua","Nyeri",
    "Samburu","Siaya","Taita Taveta","Tana River","Tharaka-Nithi","Trans Nzoia","Turkana",
    "Uasin Gishu","Vihiga","Wajir","West Pokot"
]

# Common alias fixes to match official spellings
ALIASES = {
    "Homabay": "Homa Bay",
    "Tharaka Nithi": "Tharaka-Nithi",
    "Elgeyo Marakwet": "Elgeyo-Marakwet",
    "Murang A": "Murang'A",
    "Murang'a": "Murang'A",
}

# Settlement / sub-area -> parent county (explicit, unambiguous)
AREA_TO_COUNTY = {
    # Nairobi settlements / estates
    "Dandora": "Nairobi",
    "Githurai": "Nairobi",
    "Kangemi": "Nairobi",
    "Kawangware": "Nairobi",
    "Kayole": "Nairobi",
    "Kibra": "Nairobi",
    "Mathare": "Nairobi",
    "Mukuru": "Nairobi",

    # Kisumu settlements
    "Kondele": "Kisumu",
    "Obunga": "Kisumu",

    # Mombasa neighborhood labels
    "Mwembe Tayari": "Mombasa",
    "Bangladesh": "Mombasa",  # informal settlement label in this IPC extract

    # Naming variants
    "Taita": "Taita Taveta",
    "Lamu County": "Lamu",
    "Embu (Mbeere)": "Embu",
    "Tharaka": "Tharaka-Nithi",
    "Tharaka-Nithi": "Tharaka-Nithi",

    # Turkana sub-areas (EXPLICIT mapping as requested)
    "Turkana West": "Turkana",
    "Turkana East-Kibish-Loima": "Turkana",
    "Turkana Central": "Turkana",
    "Turkana North": "Turkana",
    "Turkana South": "Turkana",
}

def collapse_area(area: str) -> str:
    """Collapse IPC Area to a parent county name."""
    if pd.isna(area):
        return np.nan

    a = re.sub(r"\s+", " ", str(area).strip())

    # Parent county for patterns like "Marsabit - Saku"
    if " - " in a:
        a = a.split(" - ", 1)[0].strip()

    a = a.title()

    # Explicit settlement / sub-area mapping
    a = AREA_TO_COUNTY.get(a, a)

    # Alias normalization
    a = ALIASES.get(a, a)

    return a

# --- Load IPC (use existing ipc_df if already loaded) ---
if "ipc_df" in globals():
    ipc = ipc_df.copy()
else:
    # If you have DATA_DIR defined, keep this; otherwise replace with your local path.
    ipc = pd.read_csv(DATA_DIR / "ipc_ken_area_long.csv")

# Standardize Area
ipc["Area_raw"] = clean_text(ipc["Area"])

# Collapse to county
ipc["county"] = ipc["Area_raw"].apply(collapse_area)
ipc["county"] = clean_text(ipc["county"])

# Keep only official counties; others become NaN for review
county_truth = set(KENYA_47)
ipc.loc[~ipc["county"].isin(county_truth), "county"] = np.nan

# --- Diagnostics ---
print("IPC unique Areas (raw):", ipc["Area_raw"].nunique())
print("IPC unique counties (collapsed, valid only):", ipc["county"].nunique())

unmapped = ipc.loc[ipc["county"].isna(), "Area_raw"].value_counts()
if len(unmapped):
    print("\nUnmapped Areas (need mapping or removal):")
    display(unmapped.head(50))

# Validity check (do NOT assert 47 counties exist in this IPC extract)
assert ipc["county"].dropna().isin(county_truth).all(), "Found invalid county values after cleaning."

# Save for downstream use
ipc_df = ipc.copy()

IPC unique Areas (raw): 53
IPC unique counties (collapsed, valid only): 31


### 3.X WFP Food Price Processing (County Completion)

WFP prices are reported at market level and cover only some counties. Here we:
- standardize admin names and map legacy districts to modern counties
- aggregate markets to monthly county medians (per commodity)
- expand to a full 47-county panel using region → national → commodity median fills
- keep `fill_level`/`fill_score` so the model can tell observed vs imputed values


In [8]:
# ============================================================
# 3.X WFP Food Price Processing (County Completion)
# Goal: market prices -> monthly county prices + fill to 47 counties
# Uses: prices_df + KENYA_47 (from Section 3A)
# Output: wfp_full (price_filled, fill_level, fill_score)
# ============================================================

import numpy as np
import pandas as pd

def to_month_start(s: pd.Series) -> pd.Series:
    # More robust parse; reduces warnings when formats vary
    return pd.to_datetime(s, errors="coerce", infer_datetime_format=True).dt.to_period("M").dt.to_timestamp()

def clean_text(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title()

# --- WFP: clean + align ---
wfp = prices_df.copy()

wfp["month"] = to_month_start(wfp["date"])
wfp["admin1"] = clean_text(wfp["admin1"])
wfp["admin2"] = clean_text(wfp["admin2"])
wfp["commodity"] = clean_text(wfp["commodity"])

# Drop header artifacts if present
wfp = wfp[wfp["admin2"] != "#Adm2+Name"].copy()
wfp = wfp[wfp["admin1"] != "#Adm1+Name"].copy()

# Map legacy districts to modern counties
district_map = {
    "Ijara": "Garissa",
    "Moyale": "Marsabit",
    "Meru North": "Meru",
    "Meru South": "Meru",
}
wfp["county"] = clean_text(wfp["admin2"].replace(district_map))

# Use USD prices (stable across time); force numeric
PRICE_COL = "usdprice"
wfp[PRICE_COL] = pd.to_numeric(wfp[PRICE_COL], errors="coerce")

# Keep only valid rows
wfp = wfp.dropna(subset=["county", "month", "commodity", PRICE_COL]).copy()

# Keep Retail only if it exists
if "pricetype" in wfp.columns:
    wfp["pricetype"] = clean_text(wfp["pricetype"])
    if (wfp["pricetype"] == "Retail").any():
        wfp = wfp[wfp["pricetype"] == "Retail"].copy()

# County-month median price per commodity
county_prices = (
    wfp.groupby(["county", "month", "commodity"], as_index=False)[PRICE_COL]
       .median()
       .rename(columns={PRICE_COL: "price"})
)

# --- 47-county reference (from Section 3A) ---
counties_47 = sorted(set(KENYA_47))
assert len(counties_47) == 47, f"KENYA_47 should have 47 counties, got {len(counties_47)}"

# Former province mapping (must cover all 47)
county_to_region = {
    # Central
    "Kiambu":"Central","Murang'A":"Central","Nyeri":"Central","Kirinyaga":"Central","Nyandarua":"Central",
    # Coast
    "Mombasa":"Coast","Kwale":"Coast","Kilifi":"Coast","Tana River":"Coast","Lamu":"Coast","Taita Taveta":"Coast",
    # Eastern
    "Machakos":"Eastern","Makueni":"Eastern","Kitui":"Eastern","Embu":"Eastern","Meru":"Eastern",
    "Tharaka-Nithi":"Eastern","Isiolo":"Eastern","Marsabit":"Eastern",
    # Nairobi
    "Nairobi":"Nairobi",
    # North Eastern
    "Garissa":"North Eastern","Wajir":"North Eastern","Mandera":"North Eastern",
    # Nyanza
    "Kisumu":"Nyanza","Siaya":"Nyanza","Homa Bay":"Nyanza","Migori":"Nyanza","Kisii":"Nyanza","Nyamira":"Nyanza",
    # Rift Valley
    "Turkana":"Rift Valley","West Pokot":"Rift Valley","Samburu":"Rift Valley","Trans Nzoia":"Rift Valley",
    "Uasin Gishu":"Rift Valley","Elgeyo-Marakwet":"Rift Valley","Nandi":"Rift Valley","Baringo":"Rift Valley",
    "Laikipia":"Rift Valley","Nakuru":"Rift Valley","Narok":"Rift Valley","Kajiado":"Rift Valley",
    "Kericho":"Rift Valley","Bomet":"Rift Valley",
    # Western
    "Kakamega":"Western","Vihiga":"Western","Bungoma":"Western","Busia":"Western",
}

regions = pd.DataFrame({"county": counties_47})
regions["region"] = regions["county"].map(county_to_region)
missing_region = regions.loc[regions["region"].isna(), "county"].tolist()
assert not missing_region, f"Missing counties in county_to_region mapping: {missing_region}"

# --- Full grid: county × month × commodity ---
months = sorted(county_prices["month"].dropna().unique())
commodities = sorted(county_prices["commodity"].dropna().unique())

wfp_full = (
    pd.MultiIndex.from_product([counties_47, months, commodities],
                               names=["county","month","commodity"])
      .to_frame(index=False)
      .merge(regions, on="county", how="left")
      .merge(county_prices, on=["county","month","commodity"], how="left")
)

# Ensure numeric dtype for medians/fills
wfp_full["price"] = pd.to_numeric(wfp_full["price"], errors="coerce")

# --- Medians for hierarchical fill ---
region_med = (
    wfp_full.groupby(["region","month","commodity"], as_index=False)["price"]
            .median()
            .rename(columns={"price":"price_region"})
)
national_med = (
    wfp_full.groupby(["month","commodity"], as_index=False)["price"]
            .median()
            .rename(columns={"price":"price_national"})
)
commodity_med = (
    wfp_full.groupby(["commodity"], as_index=False)["price"]
            .median()
            .rename(columns={"price":"price_commodity"})
)

wfp_full = wfp_full.merge(region_med, on=["region","month","commodity"], how="left")
wfp_full = wfp_full.merge(national_med, on=["month","commodity"], how="left")
wfp_full = wfp_full.merge(commodity_med, on=["commodity"], how="left")

# Force numeric on fill sources
for c in ["price_region", "price_national", "price_commodity"]:
    wfp_full[c] = pd.to_numeric(wfp_full[c], errors="coerce")

# --- Fill (NO np.where for strings) ---
wfp_full["price_filled"] = wfp_full["price"]

# Make fill_level explicitly object dtype to avoid dtype promotion errors
wfp_full["fill_level"] = pd.Series(pd.NA, index=wfp_full.index, dtype="object")

# county observed
m = wfp_full["price_filled"].notna()
wfp_full.loc[m, "fill_level"] = "county"

# region median
m = wfp_full["price_filled"].isna() & wfp_full["price_region"].notna()
wfp_full.loc[m, "price_filled"] = wfp_full.loc[m, "price_region"]
wfp_full.loc[m, "fill_level"] = "region"

# national median
m = wfp_full["price_filled"].isna() & wfp_full["price_national"].notna()
wfp_full.loc[m, "price_filled"] = wfp_full.loc[m, "price_national"]
wfp_full.loc[m, "fill_level"] = "national"

# commodity median (last fallback)
m = wfp_full["price_filled"].isna()
wfp_full.loc[m, "price_filled"] = wfp_full.loc[m, "price_commodity"]
wfp_full.loc[m, "fill_level"] = "commodity"

# Confidence score
fill_score = {"county": 3, "region": 2, "national": 1, "commodity": 0}
wfp_full["fill_score"] = wfp_full["fill_level"].map(fill_score).astype("Int64")

# --- Checks ---
assert wfp_full["county"].nunique() == 47
assert wfp_full["price_filled"].notna().all()

print("WFP full panel:", wfp_full.shape)
print(wfp_full["fill_level"].value_counts(dropna=False))

wfp_full.head()

C:\Users\Jasho\AppData\Local\Temp\ipykernel_25140\4277403747.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True).dt.to_period("M").dt.to_timestamp()
C:\Users\Jasho\AppData\Local\Temp\ipykernel_25140\4277403747.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", infer_datetime_format=True).dt.to_period("M").dt.to_timestamp()


WFP full panel: (379995, 11)
fill_level
commodity    300800
national      56399
region        18781
county         4015
Name: count, dtype: int64


,county,month,commodity,region,price,price_region,price_national,price_commodity,price_filled,fill_level,fill_score
0,Baringo,2006-01-01,Bananas,Rift Valley,NaN,NaN,NaN,0.0900,0.0900,commodity,0
1,Baringo,2006-01-01,Beans,Rift Valley,NaN,NaN,NaN,1.1500,1.1500,commodity,0
2,Baringo,2006-01-01,Beans (Dry),Rift Valley,NaN,NaN,NaN,0.9450,0.9450,commodity,0
3,Baringo,2006-01-01,Bread,Rift Valley,NaN,NaN,NaN,0.4600,0.4600,commodity,0
4,Baringo,2006-01-01,Cabbage,Rift Valley,NaN,NaN,NaN,0.5700,0.5700,commodity,0


### 4. Build Modeling Table (IPC Target + WFP Features)

In [9]:
# ============================================================
# SECTION 4 — Build Modeling Table (IPC Target + WFP Features)
# Inputs:
#   - ipc_df   (from Section 3A; must contain `county`, `Phase`, `From`, `Percentage`, `Number`)
#   - wfp_full (from Section 3.X; must contain `county`, `month`, `commodity`, `price_filled`, `fill_score`)
# Output:
#   - model_df (county-month modeling table)
# ============================================================

import numpy as np
import pandas as pd

# ---------- 4.1 Select staple commodities ----------
# Keep a small, meaningful set of staples (edit names to match your dataset)
staples = [
    "Maize Flour",
    "Maize",
    "Beans",
    "Rice",
    "Sugar",
    "Cooking Oil",
]

# If some names don't match, inspect and edit staples
# sorted(wfp_full["commodity"].unique())[:80]

wfp_staples = wfp_full[wfp_full["commodity"].isin(staples)].copy()

print("Staples kept:", wfp_staples["commodity"].nunique())
print("Fill levels (staples):")
print(wfp_staples["fill_level"].value_counts())

# ---------- 4.2 Build county-month price features ----------
# Features:
#   - p: price level
#   - p_mom: month-over-month % change (inflation shock)
#   - p_roll3: rolling 3-month mean (trend)
#   - p_vol3: rolling 3-month std (volatility)

df = wfp_staples.sort_values(["county", "commodity", "month"]).copy()
df["p"] = df["price_filled"]

df["p_mom"] = df.groupby(["county", "commodity"])["p"].pct_change()
df["p_roll3"] = df.groupby(["county", "commodity"])["p"].transform(lambda x: x.rolling(3, min_periods=2).mean())
df["p_vol3"]  = df.groupby(["county", "commodity"])["p"].transform(lambda x: x.rolling(3, min_periods=2).std())

feat_long = df[["county", "month", "commodity", "p", "p_mom", "p_roll3", "p_vol3", "fill_score"]].copy()

# Wide format (one column per commodity-feature)
feat_wide = feat_long.pivot_table(
    index=["county", "month"],
    columns="commodity",
    values=["p", "p_mom", "p_roll3", "p_vol3"],
    aggfunc="mean"
)

# Flatten columns: p__Maize_Flour
feat_wide.columns = [f"{a}__{b}".replace(" ", "_") for a, b in feat_wide.columns]
feat_wide = feat_wide.reset_index()

# Overall imputation quality for that county-month (mean across staples)
fill_q = (
    feat_long.groupby(["county", "month"], as_index=False)["fill_score"]
             .mean()
             .rename(columns={"fill_score": "wfp_fill_score_mean"})
)
feat_wide = feat_wide.merge(fill_q, on=["county", "month"], how="left")

print("\nFeature table shape:", feat_wide.shape)
feat_wide.head()

# ---------- 4.3 Build IPC Phase 3+ target (county-month) ----------
ipc = ipc_df.copy()

# Align IPC timeline to month start (use 'From' date)
ipc["month"] = pd.to_datetime(ipc["From"], errors="coerce").dt.to_period("M").dt.to_timestamp()

# Keep Phase 3+ rows only
ipc_3p = ipc[ipc["Phase"].astype(str).str.strip() == "3+"].copy()

# Numeric targets (keep both; choose one later depending on modeling)
ipc_3p["target_3p_pct"] = pd.to_numeric(ipc_3p["Percentage"], errors="coerce")
ipc_3p["target_3p_num"] = pd.to_numeric(ipc_3p["Number"], errors="coerce")

# Collapse to one row per county-month
target = (
    ipc_3p.groupby(["county", "month"], as_index=False)
          .agg(
              target_3p_pct=("target_3p_pct", "mean"),
              target_3p_num=("target_3p_num", "mean")
          )
)

print("\nTarget table shape:", target.shape)
target.head()

# ---------- 4.4 Merge target + WFP features ----------
model_df = target.merge(feat_wide, on=["county", "month"], how="left")

print("\nModel table shape:", model_df.shape)

# Quick missingness check: share of rows with ALL price level columns missing
price_level_cols = [c for c in model_df.columns if c.startswith("p__")]
all_missing_rate = model_df[price_level_cols].isna().all(axis=1).mean() if price_level_cols else np.nan
print("Rows missing all WFP price levels:", round(all_missing_rate, 3))

# Show a sample
model_df.sort_values(["county", "month"]).head(10)

Staples kept: 5
Fill levels (staples):
fill_level
commodity    44274
national      5955
region        3414
county         642
Name: count, dtype: int64

Feature table shape: (10857, 23)

Target table shape: (614, 4)

Model table shape: (614, 25)
Rows missing all WFP price levels: 0.075


,county,month,target_3p_pct,target_3p_num,p__Beans,p__Maize,p__Maize_Flour,p__Rice,p__Sugar,p_mom__Beans,p_mom__Maize,p_mom__Maize_Flour,p_mom__Rice,p_mom__Sugar,p_roll3__Beans,p_roll3__Maize,p_roll3__Maize_Flour,p_roll3__Rice,p_roll3__Sugar,p_vol3__Beans,p_vol3__Maize,p_vol3__Maize_Flour,p_vol3__Rice,p_vol3__Sugar,wfp_fill_score_mean
0,Baringo,2019-07-01,0.1500,105555.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Baringo,2019-08-01,0.2000,140740.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,Baringo,2020-02-01,0.0500,33339.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,Baringo,2020-04-01,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,Baringo,2020-08-01,0.0500,33339.0000,1.1500,0.4800,0.5700,0.9200,0.9900,0.0000,0.0323,-0.0806,0.0000,-0.1161,1.1500,0.4700,0.6033,0.9200,1.0767,0.0000,0.0087,0.0289,0.0000,0.0751,0.6000
5,Baringo,2020-10-01,0.0000,0.0000,1.1500,0.4650,0.6200,0.9200,1.1200,0.0000,0.0000,0.0000,0.0000,0.0000,1.1500,0.4700,0.6033,0.9200,1.0767,0.0000,0.0087,0.0289,0.0000,0.0751,0.0000
6,Baringo,2021-02-01,0.1000,66678.0000,1.1500,0.4050,0.6200,0.9100,1.1000,0.0000,-0.0122,-0.0312,0.0000,0.0092,1.1500,0.4267,0.6267,0.9133,1.1033,0.0000,0.0333,0.0115,0.0058,0.0153,1.6000
7,Baringo,2021-03-01,0.1500,100017.0000,1.1500,0.3800,0.5900,0.9500,1.0900,0.0000,-0.0617,-0.0484,0.0440,-0.0091,1.1500,0.3983,0.6167,0.9233,1.0933,0.0000,0.0161,0.0252,0.0231,0.0058,1.6000
8,Baringo,2021-07-01,0.2000,133356.0000,1.1500,0.4600,0.5600,0.9300,1.0800,0.0000,0.3939,-0.1111,0.0000,0.1613,1.1500,0.3733,0.5833,0.9300,1.0233,0.0000,0.0751,0.0404,0.0000,0.0814,2.0000
9,Baringo,2021-11-01,0.1000,66678.0000,1.1500,0.4200,0.5800,1.0050,1.0700,0.0000,0.1351,-0.0569,0.1824,0.0094,1.1500,0.3917,0.5950,0.9217,1.0633,0.0000,0.0257,0.0180,0.0782,0.0058,1.6000
